<br>
<img src="data/logoMoSEF.jpeg" width=90px align="left">
<div align="right">Enseignant : Hager Oueslati</div>
<div align="right">Réalisé par : Gaoussou DIAKITE et Eunice KOFFI</div>
<div align="right">Année : 2022/2023</div><br><br><br>
<div align="center"><span style="font-family:Lucida Caligraphy;font-size:32px;color:darkgreen">Master 2 Modélistaion Statistiques Economiques et Financières</span></div><br>
<div align="center"><span style="font-family:Lucida Caligraphy;font-size:28px;color:#e60000">Projet Python avancé</span></div><br><br>
<hr>

<br>

<div align="center"><span style="font-family:Lucida Caligraphy;font-size:32px;color:darkgreen"> Features engineering </span></div><br>
<hr/>

# Importation des bibliothèques et modules

In [ ]:
#établir la connexion avec google drive 
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install pyspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install findspark

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import findspark
from pyspark import  HiveContext , SparkContext


In [ ]:
from pyspark.sql import functions as F
from pyspark.sql.functions import col, count, expr, explode, regexp_extract, regexp_replace, substring, when
from pyspark.sql.window import Window

In [ ]:
sc = SparkContext()
sqlContext = HiveContext(sc)

# Importation de la base de données

In [ ]:
nexity_json = sc.wholeTextFiles('/content/drive/MyDrive/Projet Python/Nexity_2.json')
type(nexity_json)

pyspark.rdd.RDD

In [ ]:
nexity_json.take(1)
nexity_json.map(lambda x : x[0]).take(1)
nexity_json.map(lambda x : x[1]).take(1)
nexity_json = nexity_json.map(lambda x : x[1])

In [ ]:
data = sqlContext.read.json(nexity_json)
data.show()

+--------------------+--------------------+-----------+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+---------------+----------+--------------------+----------------+------+
|         Amenagement|              Bareme|Code_postal|         Copropriete|    Departement|         Description|           Essentiel|            Logement|        Loyer_Charge|     Prix_loyer|Superficie|               Titre|           Types|field1|
+--------------------+--------------------+-----------+--------------------+---------------+--------------------+--------------------+--------------------+--------------------+---------------+----------+--------------------+----------------+------+
|Pièce(s)1WC sépar...|Loyer :650 CC/moi...|      31000|Charges courantes...|       Toulouse|Plein centre vill...|Type de BienAppar...|- Appartement stu...|Loyer (CC/ mois)6...|  650 € CC/mois|     22 m²|Appartement à lou...|location meublée|     0|
|Piè

In [ ]:
data.printSchema()

root
 |-- Amenagement: string (nullable = true)
 |-- Bareme: string (nullable = true)
 |-- Code_postal: string (nullable = true)
 |-- Copropriete: string (nullable = true)
 |-- Departement: string (nullable = true)
 |-- Description: string (nullable = true)
 |-- Essentiel: string (nullable = true)
 |-- Logement: string (nullable = true)
 |-- Loyer_Charge: string (nullable = true)
 |-- Prix_loyer: string (nullable = true)
 |-- Superficie: string (nullable = true)
 |-- Titre: string (nullable = true)
 |-- Types: string (nullable = true)
 |-- field1: string (nullable = true)



In [ ]:
print("La base de données contients", data.count(), "lignes et", len(data.columns), "colonnes.")

La base de données contients 1032 lignes et 14 colonnes.


# Construction de la base de données

Nous constatons que nous avons des colonnes qui contiennent des informations complètes que d'autres et que d'une colonne, nous pouvons tirer des informations valables qui remplaceront les valeurs manquantes d'une autre colonne.

## Sélection de colonnes

In [ ]:
data = data.select('Titre', 'Amenagement','Bareme','Copropriete', 'Description', 'Essentiel', 'Loyer_Charge', 'Code_postal', 'Departement')

In [ ]:
data.show(truncate=False)

+------------------------------------------------------------+---------------------------------------------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+-----------------------------------------------------------------------------

## Création de variables

A partir des premières variables sélectionnées, nous allons recontruire les variables dont nous souhaitons disposer pour nos analyses.

## Titre

De cette variable, nous allons extraire : le type du logement, la localisation, le loyer, la superficie et le nombre de pièces.

In [ ]:
# Extraction du type du logement 
L1 = F.split(data['Titre'], 'à')
data = data.withColumn('Type',L1.getItem(0))

# Extraction de la localisation
L2 = F.split(data['Titre'], ':')
data = data.withColumn('var1',L2.getItem(1))
L3 = F.split(data['var1'], '•')

data = data.withColumn('Dep',L3.getItem(0))

# Extraction de la superficie
data = data.withColumn('Sperficie',L3.getItem(1))
L4 = F.split(data['Sperficie'], ' ')
data = data.withColumn('Superficie',L4.getItem(0)) # Je ne garde que les dimensions (les chiffres)

# Extraction du nombre de pièces
data = data.withColumn('piece',L3.getItem(2))
data = data.withColumn('piece', regexp_replace('piece', 'studio / T1', '1 pièce')) # Les studio / T1 ont une seule pièce
L5 = F.split(data['piece'], ' ')
data = data.withColumn('nb_piece',L5.getItem(0)) # Je garde que les chiffres

## Loyer_Charge

Comme son titre l'indique, nous allons extraire de cette colonne le loyer et les charges pour chaque logement. <br/> Chaque cellule contient un ou deux montants. Le premier concerne le loyer et le deuxième, les charges. Quand nous avons un seul montant, celui-ci correspond au loyer.

In [ ]:
# Extraction du loyer
L6 = F.regexp_extract('Loyer_Charge', r'(\d+)',1)
data = data.withColumn('loyer', L6)

# Extraction de la charge
L7 = F.regexp_extract('Loyer_Charge', r'(\d+)Charges/mois(\d+)', 2)
data = data.withColumn('charges', L7)

## Bareme

Cette colonne contient des informations nous disposons déjà comme le loyer, les charges et bien d'autres. Elle contient également d'autres informations comme les conditions d'attribution du logement. <br/> Nous récupérons de cette colonne uniquement le montant à payer pour le dépôt de la garantie.

In [ ]:
# Extraction de la garantie
L8 = F.regexp_extract('Bareme', r'Dépôt de garantie :(\d+)', 1)
data = data.withColumn('Garantie', L8)

## Amenagement

Cette colonne contient quelques informations sur le logement. Ces informations ne sont pas toujours disponibles pour tous les logements. Avec l'utlisation des regex nous aurons donc des valeurs manquantes pour les éléments qui n'y figurent pas.

In [ ]:
# WC séparé  (Oui/Non)
L9 = F.regexp_extract('Amenagement', r'WC séparé(\w{3})', 1)
data = data.withColumn('WC_separe', L9)

In [ ]:
# Type de chauffage (Collectif / Individuel)
L10 = F.regexp_extract('Amenagement', r'Chauffage(\w+)', 1)
data = data.withColumn('Chauffage', L10)

## Essentiel 

Nous obtiendrons, à partir de cette variable, des informations complémentaires sur les caractéristiques des logements.

In [ ]:
# Présence d'un terrain extérieur
L11 = F.regexp_extract('Essentiel', r'Terrain extérieur(\w{3})', 1)
data = data.withColumn('Terrain_extérieur', L11)

# Présence d'une terrase
L12 = F.regexp_extract('Essentiel', r'Terrasse(\w{3})', 1)
data = data.withColumn('Terrasse', L12)

# Présence d'un balcon
L13 = F.regexp_extract('Essentiel', r'Balcon(\w{3})', 1)
data = data.withColumn('Balcon', L13)

# Disponibilité de place de stationnement
L14 = F.regexp_extract('Essentiel', r'stationnement(\w{3})', 1)
data = data.withColumn('stationnement', L14)

## Copropiété

Cette colonne également nous permet d'avoir plus d'informations sur les caractéristiques communes aux logements d'un immeuble ou encore les informations relatives à l'emplacement du logement. <br/>
Notons que certaines informations seront manquantes car elles ne concernent qu'un type de logement particulier.

In [ ]:
# Numéro d'étage
L15 = F.regexp_extract('Copropriete', r'Etage(\d{1,})', 1)
data = data.withColumn('Etage', L15)

# Présence d'ascenceur
L16 = F.regexp_extract('Copropriete', r'Ascenseur(\w{3})', 1)
data = data.withColumn('Ascenseur', L16)

# Présence de cave
L17 = F.regexp_extract('Copropriete', r'Cave(\w{3})', 1)
data = data.withColumn('Cave', L17)

# Disponibilité d'un interphone
L18 = F.regexp_extract('Copropriete', r'Interphone(\w{3})', 1)
data = data.withColumn('Interphone', L18)

# Présence d'un gardien
L19 = F.regexp_extract('Copropriete', r'Gardien(\w{3})', 1)
data = data.withColumn('Gardien', L19)

# Présence d'un digicode
L20 = F.regexp_extract('Copropriete', r'Digicode(\w{3})', 1)
data = data.withColumn('Digicode', L20)

## Code Postal

Nous avons remarqué que pour certaines observations, l'informations concernant le code postal est indisponible dans une colonne mais présente dans une autre. Nous allons donc récupéré l'information de cette colonne.

In [ ]:
data = data.withColumn("Code_postal", expr("CASE WHEN Code_postal == 'NA' THEN Departement ELSE Code_postal END"))
data = data.withColumn('Code_dep', substring('Code_postal', 1,2))

## Réorganisation de la base de données

Après avoir obtenu les variables dont nous avons besoin, nous supprimons les anciennes ainsi que les variables intermédiaires construites.

In [ ]:
# Supression des colonnes 
data = data.drop(*('Titre','Amenagement','Bareme','copropriete','Essentiel','Loyer_charge','var1','Sperficie', 'piece', 'Departement'))

# Réorganisation de l'ordre des variables
data = data.select('Type', 'Dep', 'Code_dep', 'Code_postal', 'loyer', 'charges', 'Superficie', 'nb_piece', 'Garantie', 'Etage', 'Ascenseur', 'WC_separe',
                   'Cave', 'Interphone', 'Gardien', 'Digicode', 'Terrain_extérieur', 'Terrasse', 'Balcon', 'stationnement','Chauffage', 'Description')

In [ ]:
# Dimensions de la base de données
print("La base de données contients", data.count(), "lignes et", len(data.columns), "colonnes.")

La base de données contients 1032 lignes et 22 colonnes.


In [ ]:
data.show(truncate=False)

+---------------------+----------------+--------+-----------+-----+-------+----------+--------+--------+-----+---------+---------+----+----------+-------+--------+-----------------+--------+------+-------------+----------+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

# Traitement de la base de données

In [ ]:
# Vérification du type des variables
data.printSchema()

root
 |-- Type: string (nullable = true)
 |-- Dep: string (nullable = true)
 |-- Code_dep: string (nullable = true)
 |-- Code_postal: string (nullable = true)
 |-- loyer: string (nullable = true)
 |-- charges: string (nullable = true)
 |-- Superficie: string (nullable = true)
 |-- nb_piece: string (nullable = true)
 |-- Garantie: string (nullable = true)
 |-- Etage: string (nullable = true)
 |-- Ascenseur: string (nullable = true)
 |-- WC_separe: string (nullable = true)
 |-- Cave: string (nullable = true)
 |-- Interphone: string (nullable = true)
 |-- Gardien: string (nullable = true)
 |-- Digicode: string (nullable = true)
 |-- Terrain_extérieur: string (nullable = true)
 |-- Terrasse: string (nullable = true)
 |-- Balcon: string (nullable = true)
 |-- stationnement: string (nullable = true)
 |-- Chauffage: string (nullable = true)
 |-- Description: string (nullable = true)



Nous constatons que toutes les variables sont au format str.<br/>
Nous allons changer le type de variable.

In [ ]:
# Importation du type de données 
from pyspark.sql.types import IntegerType ,StringType, FloatType

In [ ]:
# Colonnes à mettre en int
data = data.withColumn('nb_piece',data.nb_piece.cast(IntegerType()))

# Variables à mettre en float
data = data.withColumn('loyer',data.loyer.cast(FloatType()))
data = data.withColumn('charges',data.charges.cast(FloatType()))
data = data.withColumn('Superficie',data.Superficie.cast(FloatType()))
data = data.withColumn('Garantie',data.Garantie.cast(FloatType()))

In [ ]:
# Vérification du type des variables
data.printSchema()

root
 |-- Type: string (nullable = true)
 |-- Dep: string (nullable = true)
 |-- Code_dep: string (nullable = true)
 |-- Code_postal: string (nullable = true)
 |-- loyer: float (nullable = true)
 |-- charges: float (nullable = true)
 |-- Superficie: float (nullable = true)
 |-- nb_piece: integer (nullable = true)
 |-- Garantie: float (nullable = true)
 |-- Etage: string (nullable = true)
 |-- Ascenseur: string (nullable = true)
 |-- WC_separe: string (nullable = true)
 |-- Cave: string (nullable = true)
 |-- Interphone: string (nullable = true)
 |-- Gardien: string (nullable = true)
 |-- Digicode: string (nullable = true)
 |-- Terrain_extérieur: string (nullable = true)
 |-- Terrasse: string (nullable = true)
 |-- Balcon: string (nullable = true)
 |-- stationnement: string (nullable = true)
 |-- Chauffage: string (nullable = true)
 |-- Description: string (nullable = true)



In [ ]:
data.show()

+--------------------+----------------+--------+-----------+------+-------+----------+--------+--------+-----+---------+---------+----+----------+-------+--------+-----------------+--------+------+-------------+----------+--------------------+
|                Type|             Dep|Code_dep|Code_postal| loyer|charges|Superficie|nb_piece|Garantie|Etage|Ascenseur|WC_separe|Cave|Interphone|Gardien|Digicode|Terrain_extérieur|Terrasse|Balcon|stationnement| Chauffage|         Description|
+--------------------+----------------+--------+-----------+------+-------+----------+--------+--------+-----+---------+---------+----+----------+-------+--------+-----------------+--------+------+-------------+----------+--------------------+
|        Appartement |        Toulouse|      31|      31000| 650.0|   30.0|      22.0|       1|  1240.0|    1|      Oui|      Oui| Oui|       Oui|    Oui|     Oui|              Non|     Oui|   Non|          Non|          |Plein centre vill...|
|        Appartement |  

# Exportation de la base de données

Nous exportons la base de données en csv.

In [ ]:
data.write.options(header='True', delimiter=',').csv("/content/drive/MyDrive/Projet Python/données.csv")

<ul>
    <li><a href="0. Sommaire et résumé.ipynb">Retourner au sommaire</a></li>
    <li><a href="3. Visualisation.ipynb">Aller à la partie 3</a></li>
    <li><a href="https://eunicekoffi-projetpython-app-wuniri.streamlit.app/">Lien de l'application</a></li>
</ul>